In [1]:
run Keypoints_RCNN.ipynb

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-2kkdawtw because the default path (/home/jetbot/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


1.7.0
CUDA available: True
cuDNN version: 8000


In [2]:
run 2_Calculating_Angles.ipynb

In [4]:
run 3_Reading_RPLidar.ipynb

In [40]:
run 4_Stage_Estimation.ipynb

In [51]:
class Environment():
    
    '''
    This wrapper works as an RL-Environment for the Jetson Nano.
    Arguments are: 
        model  -> pytorch model trained in Unity.
        angles -> list of angles required to measure
        FOV    -> How wide the camera lens is. Default = 160
    '''
    
    def __init__(self, model, angles, FOV=160):
        self.previous_readings = {x:0 for x in angles} 
        self.angles = angles
        self.model = model
        self.FOV = FOV
        
        
    def calculate_angle_and_phase(self):
        keypoints, image, counts, objects, peaks = execute({'new': camera.value})
        angle, keypoints = calculate_angle(WIDTH, keypoints, self.FOV)
        phase = self.calculate_phase(keypoints)
        return phase, angle
    
    
    def calculate_phase(self, keypoints):
        phase = estimating_phase(keypoints)
        return phase
    
    
    def read_lidar(self):
        self.previous_readings = read_lidar_wrapper(self.angles,self.previous_readings)
        
    
    def observe(self):
        self.read_lidar()
        phase, angle = self.calculate_angle_and_phase()
        
        if isinstance(angle, dict):
            angles = list(angle.values())
        
        else: angles = [-1]
        
        observation = phase + angle + list(self.previous_readings.values())
        return observation
    
    
    def sample_action(self, observation):
        observation = torch.Tensor(observation).cuda()
        hidden,_ = self.model.network_body(vis_inputs=[0],vec_inputs=[observation])
        distribution = self.model.distribution(hidden)
        action = distribution.sample()
        return action

In [52]:
model = torch.load("MobileRobot-16325.pth")
model.eval()

SeparateActorCritic(
  (network_body): NetworkBody(
    (visual_processors): ModuleList()
    (vector_processors): ModuleList(
      (0): VectorInput(
        (normalizer): Normalizer()
      )
    )
    (linear_encoder): LinearEncoder(
      (seq_layers): Sequential(
        (0): Linear(in_features=22, out_features=256, bias=True)
        (1): Swish()
        (2): Linear(in_features=256, out_features=256, bias=True)
        (3): Swish()
        (4): Linear(in_features=256, out_features=256, bias=True)
        (5): Swish()
      )
    )
  )
  (distribution): GaussianDistribution(
    (mu): Linear(in_features=256, out_features=2, bias=True)
  )
  (critic): ValueNetwork(
    (network_body): NetworkBody(
      (visual_processors): ModuleList()
      (vector_processors): ModuleList(
        (0): VectorInput(
          (normalizer): Normalizer()
        )
      )
      (linear_encoder): LinearEncoder(
        (seq_layers): Sequential(
          (0): Linear(in_features=22, out_features=256, 

In [59]:
angles = [x for x in range(0,180,10)]

In [58]:
env = Environment(model, angles)
observation = env.observe()
print(observation)

[1, 0, 0, -1, 568, 560, 1687, 0, 2014, 0, 386, 0, 542, 654, 459, 433, 507, 482, 511, 519, 334, 0]


In [57]:
action = env.sample_action(observation)
action

tensor([[-0.2607, -0.3771]], grad_fn=<AddBackward0>)